Sample code for page rank. it use a full dimention matrix.

In [98]:
%config IPCompleter.greedy=True

In [89]:
import numpy as np
from scipy.sparse import csc_matrix

def pageRank(G, s = .85, maxerr = .0001):
    """
    Computes the pagerank for each of the n states
    Parameters
    ----------
    G: matrix representing state transitions
       Gij is a binary value representing a transition from state i to j.
    s: probability of following a transition. 1-s probability of teleporting
       to another state.
    maxerr: if the sum of pageranks between iterations is bellow this we will
            have converged.
    """
    n = G.shape[0]

    # transform G into markov matrix A
    A = csc_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
# calculate
        for i in range(0,n):
# inlin
            Ai = np.array(A[:,i].todense())[:,0]
            # account for sink states
            Di = sink / float(n)
            # account for teleportation to state i
            Ei = np.ones(n) / float(n)

            r[i] = ro.dot( Ai*s + Di*s + Ei*(1-s) )

    # return normalized pagerank
    return r/float(sum(r))

if __name__=='__main__':
    # Example extracted from 'Introduction to Information Retrieval'
    G = np.array([[0,0,1,0,0,0,0],
                  [0,1,1,0,0,0,0],
                  [1,0,1,1,0,0,0],
                  [0,0,0,1,1,0,0],
                  [0,0,0,0,0,0,1],
                  [0,0,0,0,0,1,1],
                  [0,0,0,1,1,0,1]])
    print(pageRank(G,s=.86))

[0.12727557 0.03616954 0.12221594 0.22608452 0.28934412 0.03616954
 0.16274076]


In [100]:
outDeg = dict()
inDeg = dict()
filename = "outDeg.txt"
with open(filename,'r') as f:
    for line in f.readlines():
        parts = line.split()
        outDeg[parts[0]] = int(parts[1])

filename = "inDeg.txt"
with open(filename,'r') as f:
    for line in f.readlines():
        parts = line.split()
        inDeg[parts[0]] = int(parts[1])
        

Main PR algorithm. Using sparse matrix.

In [142]:
import numpy as np
from  scipy import sparse
filename = "enumsite.txt"
NODES = 2287
# EDGES = 73077
EDGES = 14275
def dataset2dok():

    with open(filename,'r') as f:
        dokm = sparse.dok_matrix((NODES,NODES),dtype=np.bool)
        for line in f.readlines():
            parts = line.split()
            if(outDeg[parts[1]] > 1):
                origin, destiny = (int(x)-1 for x in line.split())
                dokm[destiny,origin]=True
    return(dokm.tocsr())
%time dok_m = dataset2dok()

Wall time: 350 ms


In [143]:
def dataset2csr():
    row = []
    col = []    
    with open(filename,'r') as f:
        for line in f.readlines():
            parts = line.split()
            if(outDeg[parts[1]] > 1):
                origin, destiny = (int(x)-1 for x in line.split())
                row.append(destiny)
                col.append(origin)
    return(sparse.csr_matrix(([True]*EDGES,(row,col)),shape=(NODES,NODES)))

%time csr_m = dataset2csr()

Wall time: 155 ms


In [144]:
import sys

print("The size in memory of the adjacency matrix is {0} MB".format(
    (sys.getsizeof(csr_m.shape)+
    csr_m.data.nbytes+
    csr_m.indices.nbytes+
    csr_m.indptr.nbytes)/(1024.0**2)
))

The size in memory of the adjacency matrix is 0.05390739440917969 MB


In [145]:
def csr_save(filename,csr):
    np.savez(filename,
        nodes=csr.shape[0],
        edges=csr.data.size,
        indices=csr.indices,
        indptr =csr.indptr
    )

def csr_load(filename):
    loader = np.load(filename)
    edges = int(loader['edges'])
    nodes = int(loader['nodes'])
    return sparse.csr_matrix(
        (np.bool_(np.ones(edges)), loader['indices'], loader['indptr']),
        shape = (nodes,nodes)
    )

In [146]:
DATASET_NATIVE = 'dataset-native.npz'
csr_save(DATASET_NATIVE,csr_m)
%time csr = csr_load(DATASET_NATIVE)

Wall time: 5.98 ms


In [147]:
def compute_PageRank(G, beta=0.85, epsilon=10**-6):
    '''
    Efficient computation of the PageRank values using a sparse adjacency 
    matrix and the iterative power method.
    
    Parameters
    ----------
    G : boolean adjacency matrix. np.bool8
        If the element j,i is True, means that there is a link from i to j.
    beta: 1-teleportation probability.
    epsilon: stop condition. Minimum allowed amount of change in the PageRanks
        between iterations.

    Returns
    -------
    output : tuple
        PageRank array normalized top one.
        Number of iterations.

    '''    
    #Test adjacency matrix is OK
    n,_ = G.shape
    assert(G.shape==(n,n))
    #Constants Speed-UP
    deg_out_beta = G.sum(axis=0).T/beta #vector
    #Initialize
    ranks = np.ones((n,1))/n #vector
    time = 0
    flag = True
    while flag:        
        time +=1
        with np.errstate(divide='ignore'): # Ignore division by 0 on ranks/deg_out_beta
            new_ranks = G.dot((ranks/deg_out_beta)) #vector
        #Leaked PageRank
        new_ranks += (1-new_ranks.sum())/n
        #Stop condition
        if np.linalg.norm(ranks-new_ranks,ord=1)<=epsilon:
            flag = False        
        ranks = new_ranks
    return(ranks, time)

In [148]:
print('==> Computing PageRank')
%time pr,iters = compute_PageRank(csr)
print('\nIterations: {0}'.format(iters))
print('Element with the highest PageRank: {0}'.format(np.argmax(pr)+1))

==> Computing PageRank
Wall time: 19 ms

Iterations: 47
Element with the highest PageRank: 40


In [149]:
mapper = dict()
with open("sitemap.txt",'r') as f:
    for line in f.readlines():
        parts = line.split()
        mapper[parts[1]] = parts[0]

In [150]:
siteranks = []
for ind, rank in enumerate(pr.tolist()):
    siteranks.append((rank[0],mapper[str(ind+1)]))

In [154]:
siteranks.sort(reverse=True)
siteranks

[(0.01815818436556825,
  'https://www.coursera.org/specializations/data-science-python'),
 (0.017199694437120824,
  'https://www.coursera.org/specializations/python-3-programming'),
 (0.016065356982371715, 'https://www.coursera.org/specializations/python'),
 (0.009049175348151068,
  'https://www.coursera.org/specializations/introduction-scripting-in-python'),
 (0.009038827005663868,
  'https://www.coursera.org/specializations/programming-in-python'),
 (0.008158492179077834,
  'https://www.coursera.org/specializations/statistics-with-python'),
 (0.007811279734533315, 'https://www.coursera.org/learn/python-data-analysis'),
 (0.00720733109812115, 'https://www.coursera.org/learn/python-network-data'),
 (0.007180115929983984, 'https://www.coursera.org/learn/python-databases'),
 (0.006860178652829741, 'https://www.coursera.org/learn/python-plotting'),
 (0.006623226217407892, 'https://www.coursera.org/learn/python-data'),
 (0.006567945057932616,
  'https://www.coursera.org/learn/python-machin

In [152]:
out = open("PR.txt", "w")
with open(filename,'r') as f:
    for rank, url in siteranks:
        out.write(str(rank) + " " + str(url) + "\n")
out.close()

Another PR algorithm

In [93]:
def pageRank2(G, alpha=0.85, personalization=None, 
             max_iter=100, tol=1.0e-6, nstart=None, weight='weight', 
             dangling=None): 
    """Return the PageRank of the nodes in the graph. 
  
    PageRank computes a ranking of the nodes in the graph G based on 
    the structure of the incoming links. It was originally designed as 
    an algorithm to rank web pages. 
  
    Parameters 
    ---------- 
    G : graph 
      A NetworkX graph.  Undirected graphs will be converted to a directed 
      graph with two directed edges for each undirected edge. 
  
    alpha : float, optional 
      Damping parameter for PageRank, default=0.85. 
  
    personalization: dict, optional 
      The "personalization vector" consisting of a dictionary with a 
      key for every graph node and nonzero personalization value for each node. 
      By default, a uniform distribution is used. 
  
    max_iter : integer, optional 
      Maximum number of iterations in power method eigenvalue solver. 
  
    tol : float, optional 
      Error tolerance used to check convergence in power method solver. 
  
    nstart : dictionary, optional 
      Starting value of PageRank iteration for each node. 
  
    weight : key, optional 
      Edge data key to use as weight.  If None weights are set to 1. 
  
    dangling: dict, optional 
      The outedges to be assigned to any "dangling" nodes, i.e., nodes without 
      any outedges. The dict key is the node the outedge points to and the dict 
      value is the weight of that outedge. By default, dangling nodes are given 
      outedges according to the personalization vector (uniform if not 
      specified). This must be selected to result in an irreducible transition 
      matrix (see notes under google_matrix). It may be common to have the 
      dangling dict to be the same as the personalization dict. 
  
    Returns 
    ------- 
    pagerank : dictionary 
       Dictionary of nodes with PageRank as value 
  
    Notes 
    ----- 
    The eigenvector calculation is done by the power iteration method 
    and has no guarantee of convergence.  The iteration will stop 
    after max_iter iterations or an error tolerance of 
    number_of_nodes(G)*tol has been reached. 
  
    The PageRank algorithm was designed for directed graphs but this 
    algorithm does not check if the input graph is directed and will 
    execute on undirected graphs by converting each edge in the 
    directed graph to two edges. 
  
      
    """
    if len(G) == 0: 
        return {} 
  
    if not G.is_directed(): 
        D = G.to_directed() 
    else: 
        D = G 
  
    # Create a copy in (right) stochastic form 
    W = nx.stochastic_graph(D, weight=weight) 
    N = W.number_of_nodes() 
  
    # Choose fixed starting vector if not given 
    if nstart is None: 
        x = dict.fromkeys(W, 1.0 / N) 
    else: 
        # Normalized nstart vector 
        s = float(sum(nstart.values())) 
        x = dict((k, v / s) for k, v in nstart.items()) 
  
    if personalization is None: 
  
        # Assign uniform personalization vector if not given 
        p = dict.fromkeys(W, 1.0 / N) 
    else: 
        missing = set(G) - set(personalization) 
        if missing: 
            raise NetworkXError('Personalization dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing) 
        s = float(sum(personalization.values())) 
        p = dict((k, v / s) for k, v in personalization.items()) 
  
    if dangling is None: 
  
        # Use personalization vector if dangling vector not specified 
        dangling_weights = p 
    else: 
        missing = set(G) - set(dangling) 
        if missing: 
            raise NetworkXError('Dangling node dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing) 
        s = float(sum(dangling.values())) 
        dangling_weights = dict((k, v/s) for k, v in dangling.items()) 
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0] 
  
    # power iteration: make up to max_iter iterations 
    for _ in range(max_iter): 
        xlast = x 
        x = dict.fromkeys(xlast.keys(), 0) 
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes) 
        for n in x: 
  
            # this matrix multiply looks odd because it is 
            # doing a left multiply x^T=xlast^T*W 
            for nbr in W[n]: 
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight] 
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n] 
  
        # check convergence, l1 norm 
        err = sum([abs(x[n] - xlast[n]) for n in x]) 
        if err < N*tol: 
            return x 
    raise NetworkXError('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter) 

import networkx as nx 
with open(filename,'r') as f:
    edgelist = [
        tuple(int(x)-1 for x in line.split())
        for line in f.readlines()
    ] 
    
print('\n==> Building graph.')
%time G = nx.from_edgelist(edgelist, create_using=nx.DiGraph())
pr=nx.pagerank(G,0.4) 
pr_max = max(pr.items(), key= lambda x: x[1])
print('\nElement with the highest PageRank: {0}'.format(pr_max[0]+1))


==> Building graph.
Wall time: 111 ms

Element with the highest PageRank: 2


In [153]:
pr

matrix([[0.01606536],
        [0.00013425],
        [0.00013425],
        ...,
        [0.00013425],
        [0.00013425],
        [0.00013425]])